In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np

pd.set_option('display.max_columns', 500)

def train_test_split(X, y, train_idx=None, test_idx=None):
    X_train = X.loc[train_idx]
    y_train = y.loc[train_idx]
    X_test = X.loc[test_idx]
    y_test = y.loc[test_idx]
    return (X_train, y_train, X_test, y_test)

def load_split_data(suffix=None, split=False, window=14):
    if suffix==None:
        suffix='DEFAULT'
    try:
        X = pd.read_pickle(f'data/X_{suffix}.pkl')
        y = pd.read_pickle(f'data/y_{suffix}.pkl')
    except:
        X, y, _ = build_Xy(df, window=window, use_atr=True, atr_ratio=(20,5), reverse=False, debug=True)
        X.to_pickle(f'data/X_{suffix}.pkl')
        y.to_pickle(f'data/y_{suffix}.pkl')
        
    if split:
        X_train, y_train, X_test, y_test = train_test_split(X, y, X.loc['2018':'2020'].index, X.loc['2021':].index)
        return X_train, y_train, X_test, y_test
    else:
        return X, y
    
data_files = [
    {'suffix' : '20210801b',
     'model_reverse' : True,
     'model_ratio' : (2,1),
     'model_use_atr' : True,
    },
    {'suffix' : '20210801c',
     'model_reverse' : True,
     'model_ratio' : (4,2),
     'model_use_atr' : True,
    },
    {'suffix' : '20210801d',
     'model_reverse' : True,
     'model_ratio' : (4,1),
     'model_use_atr' : True,
    },
    {'suffix' : '20210801e',
     'model_reverse' : True,
     'model_ratio' : (0.005,0.0025),
     'model_use_atr' : False,
    },
    {'suffix' : '20210801f',
     'model_reverse' : True,
     'model_ratio' : (0.01,0.005),
     'model_use_atr' : False,
    },
    {'suffix' : '20210803a',
     'model_reverse' : True,
     'model_ratio' : (2,1),
     'model_use_atr' : True,
    },
    {'suffix' : '20210803b',
     'model_reverse' : True,
     'model_ratio' : (3,1),
     'model_use_atr' : True,
    },
    {'suffix' : '20210803c',
     'model_reverse' : True,
     'model_ratio' : (4,1),
     'model_use_atr' : True,
    },
    {'suffix' : '20210803d',
     'model_reverse' : True,
     'model_ratio' : (4,2),
     'model_use_atr' : True,
    },
    {'suffix' : '20210803e',
     'model_reverse' : True,
     'model_ratio' : (5,2),
     'model_use_atr' : True,
    },
    {'suffix' : '20210803f',
     'model_reverse' : True,
     'model_ratio' : (0.0025,0.00125),
     'model_use_atr' : False,
    },
    {'suffix' : '20210803g',
     'model_reverse' : True,
     'model_ratio' : (0.005,0.0025),
     'model_use_atr' : False,
    },
    {'suffix' : '20210803h',
     'model_reverse' : True,
     'model_ratio' : (0.01,0.005),
     'model_use_atr' : False,
    },
    {'suffix' : '20210803i',
     'model_reverse' : True,
     'model_ratio' : (0.01,0.0025),
     'model_use_atr' : False,
    },
    {'suffix' : '20210803j',
     'model_reverse' : True,
     'model_ratio' : (0.0075,0.0025),
     'model_use_atr' : False,
    },
]


for d in data_files:
    X_train, y_train, X_test, y_test = load_split_data(suffix=d['suffix'], split=True)
    d['X_train'] = X_train
    d['y_train'] = y_train
    d['X_test'] = X_test
    d['y_test'] = y_test

In [ ]:
# Simulate...

def get_target_stoploss(df, threshold_ratio=(0.04,0.02), use_atr=True, atr_ratio=(2,1), reverse=False):
    if not reverse:
        if use_atr:
            stop_losses = df.low-(df.atr*atr_ratio[1])
            targets = df.close+(df.atr*atr_ratio[0])
        else:
            stop_losses = df.close-df.close*threshold_ratio[1]
            targets = df.close+df.close*threshold_ratio[0]
    else:
        if use_atr:
            stop_losses = df.high+(df.atr*atr_ratio[1])
            targets = df.close-(df.atr*atr_ratio[0])
        else:
            stop_losses = df.close+df.close*threshold_ratio[1]
            targets = df.close-df.close*threshold_ratio[0]

    return targets, stop_losses

def get_decisions_and_prices(x_data, pred, info_dict):
    next_action = 1
    target = -1
    stoploss = -1
    
    if type(x_data.index) != pd.RangeIndex:
        x_data = x_data.reset_index(drop=True)
    
    if type(pred) in (pd.DataFrame, pd.Series):
        pred = pred.to_numpy().ravel()

    use_atr = info_dict['model_use_atr']
    atr_ratio = info_dict['model_ratio']
    threshold_ratio = info_dict['model_ratio']
    reverse = info_dict['model_reverse']
        
    targets, stop_losses = get_target_stoploss(x_data,
                                               use_atr=use_atr,
                                               atr_ratio=atr_ratio,
                                               threshold_ratio=threshold_ratio,
                                               reverse=reverse)
    low_prices = x_data['low'].to_numpy()
    high_prices = x_data['high'].to_numpy()
    
    # Decisions:
    # 1 = buy
    # 0 = hold (default)
    # -1 = sell
    decision = pd.Series(0, index=x_data.index)
    execution_price = pd.Series(0.0, index=x_data.index)

    i = 0
    while True:
        if i>=len(x_data):
            break
        if next_action == 1:
            # Find next buy opportunity
            try:
                next_buy_idx = np.where(pred[i:]==1)[0][0] + i
                target = targets.iloc[next_buy_idx]
                stoploss = stop_losses.iloc[next_buy_idx]
                decision.at[next_buy_idx] = 1
                execution_price.at[next_buy_idx] = x_data.loc[next_buy_idx, 'close']
                i = next_buy_idx+1
                next_action = -1
            except:
                # No more buy opportunties
                break
        else:
            # Find next sell opportunity
            try:
                if not reverse:
                    next_sell_idx = np.where((high_prices[i:]>=target) | (low_prices[i:]<=stoploss))[0][0] + i
                else:
                    next_sell_idx = np.where((low_prices[i:]<=target) | (high_prices[i:]>=stoploss))[0][0] + i
                if x_data.loc[next_sell_idx, 'low'] <= target <= x_data.loc[next_sell_idx, 'high']:
                    execution_price.at[next_sell_idx] = target
                else:
                    execution_price.at[next_sell_idx] = stoploss
                decision.at[next_sell_idx] = -1
                i = next_sell_idx+1
                next_action = 1
            except:
                # No more sell opportunties
                break

    return decision, execution_price

In [ ]:
for d in data_files:
    print(f"Inbalance for {d['suffix']}: Train: {float(d['y_train'].sum()/len(d['y_train']))} Test: {float(d['y_test'].sum()/len(d['y_test']))}")

```
Inbalance for 20210801b: Train: 0.3919901522958892 Test: 0.40973782771535583
Inbalance for 20210801c: Train: 0.2274418870949273 Test: 0.23830212234706616
Inbalance for 20210801d: Train: 0.18943471124852093 Test: 0.20149812734082398
Inbalance for 20210801e: Train: 0.4076205198671705 Test: 0.418227215980025
Inbalance for 20210801f: Train: 0.262032902019161 Test: 0.31920099875156055
Inbalance for 20210803a: Train: 0.3919901522958892 Test: 0.40973782771535583
Inbalance for 20210803b: Train: 0.27318790793541736 Test: 0.2919350811485643
Inbalance for 20210803c: Train: 0.18943471124852093 Test: 0.20149812734082398
Inbalance for 20210803d: Train: 0.2274418870949273 Test: 0.23830212234706616
Inbalance for 20210803e: Train: 0.15737241879461047 Test: 0.16179775280898875
Inbalance for 20210803f: Train: 0.1843009275163174 Test: 0.12094881398252184
Inbalance for 20210803g: Train: 0.2883029886636895 Test: 0.2633707865168539
Inbalance for 20210803h: Train: 0.262032902019161 Test: 0.31920099875156055
Inbalance for 20210803i: Train: 0.1696343371884423 Test: 0.1969538077403246
Inbalance for 20210803j: Train: 0.2200942784075728 Test: 0.23380774032459425
```

In [ ]:
trading_fees_buy = 0
trading_fees_sell = 0
trading_fees_percent = 0.1
starting_value = 1

def simulate(in_df):
    df = in_df.copy()
    df['value'] = 0.0
    value = starting_value
    fee_multiplier = 1.0 - trading_fees_percent / 100

    for x,r in df.iterrows():
        if r.decision == 1 and value > 0:
            value = ((value-trading_fees_buy) * r.price) * fee_multiplier
            if value < 0:
                break
        elif r.decision == -1 and value > 0:
            value = ((value-trading_fees_sell) / r.price) * fee_multiplier
            if value < 0:
                break
        else:
            break # value is below zero
        df.loc[x,'value'] = value
    return df.value

def print_results(name, ratio, df):
    sales = df[df.decision==-1].value
    number_of_trades = len(sales)
    last_price = df[df.decision==-1].value.to_numpy()[-1]
    losing_trades = (sales.diff()<0).sum()

    print(f'''{name} - {ratio}:
    Number of trades: {number_of_trades}
    Maximum Profit: {last_price}
    Failed Trades: {losing_trades}
    ''')

for d in data_files:
    decision, execution_price = get_decisions_and_prices(d['X_test'], d['y_test'], d)
    d['X_test']['decision'] = decision.values
    d['X_test']['price'] = execution_price.values
    
    df = d['X_test'][d['X_test']['decision']!=0][['decision','price']].copy()
    df['value'] = simulate(df)
    print_results(d['suffix'], d['model_ratio'], df)

Fee = 0.1%:
```
20210801b - (2, 1):
    Number of trades: 1033
    Maximum Profit: 9843.07824744504
    Failed Trades: 2
    
20210801c - (4, 2):
    Number of trades: 368
    Maximum Profit: 1084.998178411117
    Failed Trades: 0
    
20210801d - (4, 1):
    Number of trades: 376
    Maximum Profit: 1295.2163562956428
    Failed Trades: 2
    
20210801e - (0.005, 0.0025):
    Number of trades: 3027
    Maximum Profit: 2.232238518315667
    Failed Trades: 1107
    
20210801f - (0.01, 0.005):
    Number of trades: 1179
    Maximum Profit: 13228.993600153219
    Failed Trades: 0
    
20210803a - (2, 1):            - same as 20210801b
    Number of trades: 1033
    Maximum Profit: 9843.07824744504
    Failed Trades: 2
    
20210803b - (3, 1):
    Number of trades: 578
    Maximum Profit: 3387.7900264437994
    Failed Trades: 2
    
20210803c - (4, 1):            - same as 20210801d
    Number of trades: 376
    Maximum Profit: 1295.2163562956428
    Failed Trades: 2
    
20210803d - (4, 2):            - same as 20210801c
    Number of trades: 368
    Maximum Profit: 1084.998178411117
    Failed Trades: 0
    
20210803e - (5, 2):
    Number of trades: 246
    Maximum Profit: 342.2451048032216
    Failed Trades: 0
    
20210803f - (0.0025, 0.00125):
    Number of trades: 1551
    Maximum Profit: 2.1788624605512403
    Failed Trades: 0
    
20210803g - (0.005, 0.0025):     - different from 20210801e due to decision threshold changes
    Number of trades: 1954
    Maximum Profit: 359.4414934041169
    Failed Trades: 0
    
20210803h - (0.01, 0.005):       - same as 20210801f
    Number of trades: 1179
    Maximum Profit: 13228.993600153219
    Failed Trades: 0
    
20210803i - (0.01, 0.0025):
    Number of trades: 1093
    Maximum Profit: 6620.481355927992
    Failed Trades: 0
    
20210803j - (0.0075, 0.0025):
    Number of trades: 1456
    Maximum Profit: 3126.592202688446
    Failed Trades: 0
    
```